In [16]:
#Loading liabraries and dataset
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
ticker = yf.Ticker("INFY")
df = ticker.history(start="2022-01-01", end="2024-12-31", interval="1d")


# --- Technical Indicators ---

def add_bollinger_bands(df, window=20, num_std=2):
    df['MA20'] = df['Close'].rolling(window).mean()
    df['Upper_Band'] = df['MA20'] + num_std * df['Close'].rolling(window).std()
    df['Lower_Band'] = df['MA20'] - num_std * df['Close'].rolling(window).std()
    return df

def add_macd(df):
    df['EMA12'] = df['Close'].ewm(span=12, adjust=False).mean()
    df['EMA26'] = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = df['EMA12'] - df['EMA26']
    df['signalline'] = df['MACD'].ewm(span=9, adjust=False).mean()
    return df

def add_rsi(df, window=14):
    delta = df['Close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window).mean()
    avg_loss = loss.rolling(window).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))
    return df

def add_stochastic(df, k_period=14, d_period=3):
    low_min = df['Low'].rolling(k_period).min()
    high_max = df['High'].rolling(k_period).max()
    df['%K'] = 100 * ((df['Close'] - low_min) / (high_max - low_min))
    df['%D'] = df['%K'].rolling(d_period).mean()
    return df

def add_atr(df, period=14):
    df['H-L'] = df['High'] - df['Low']
    df['H-PC'] = abs(df['High'] - df['Close'].shift(1))
    df['L-PC'] = abs(df['Low'] - df['Close'].shift(1))
    df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
    df['ATR'] = df['TR'].rolling(period).mean()
    return df

def add_vwap(df):
    df['TP'] = (df['High'] + df['Low'] + df['Close']) / 3
    df['Cumulative_TP_Volume'] = (df['TP'] * df['Volume']).cumsum()
    df['Cumulative_Volume'] = df['Volume'].cumsum()
    df['VWAP'] = df['Cumulative_TP_Volume'] / df['Cumulative_Volume']
    return df

# --- Signal Generators ---

def signal_bollinger(df):
    df['BB_SIGNAL'] = 0
    df.loc[df['Close'] < df['Lower_Band'], 'BB_SIGNAL'] = 1
    df.loc[df['Close'] > df['Upper_Band'], 'BB_SIGNAL'] = -1
    print('Bollinger Bands buy signals:', (df['BB_SIGNAL'] == 1).sum())
    print('Bollinger Bands sell signals:', (df['BB_SIGNAL'] == -1).sum())
    return df

def signal_macd(df):
    df['MACD_SIGNAL_FLAG'] = 0
    df.loc[df['MACD'] > df['signalline'], 'MACD_SIGNAL_FLAG'] = 1
    df.loc[df['MACD'] < df['signalline'], 'MACD_SIGNAL_FLAG'] = -1
    print('MACD buy signals:', (df['MACD_SIGNAL_FLAG'] == 1).sum())
    print('MACD sell signals:', (df['MACD_SIGNAL_FLAG'] == -1).sum())
    return df

def signal_rsi(df, overbought=70, oversold=30):
    df['RSI_SIGNAL'] = 0
    df.loc[df['RSI'] < oversold, 'RSI_SIGNAL'] = 1
    df.loc[df['RSI'] > overbought, 'RSI_SIGNAL'] = -1
    print('RSI buy signals:', (df['RSI_SIGNAL'] == 1).sum())
    print('RSI sell signals:', (df['RSI_SIGNAL'] == -1).sum())
    return df

def signal_stochastic(df, overbought=80, oversold=20):
    df['STOCH_SIGNAL'] = 0
    df.loc[(df['%K'] < oversold) & (df['%K'] > df['%D']), 'STOCH_SIGNAL'] = 1
    df.loc[(df['%K'] > overbought) & (df['%K'] < df['%D']), 'STOCH_SIGNAL'] = -1
    print('Stochastic buy signals:', (df['STOCH_SIGNAL'] == 1).sum())
    print('Stochastic sell signals:', (df['STOCH_SIGNAL'] == -1).sum())
    return df

def signal_atr(df, threshold=1.5):
    df['ATR_SIGNAL'] = 0
    df.loc[df['ATR'] > threshold, 'ATR_SIGNAL'] = 1
    print('ATR high volatility signals:', (df['ATR_SIGNAL'] == 1).sum())
    return df

def signal_vwap(df):
    df['VWAP_SIGNAL'] = 0
    df.loc[df['Close'] > df['VWAP'], 'VWAP_SIGNAL'] = 1
    df.loc[df['Close'] < df['VWAP'], 'VWAP_SIGNAL'] = -1
    print('VWAP buy signals:', (df['VWAP_SIGNAL'] == 1).sum())
    print('VWAP sell signals:', (df['VWAP_SIGNAL'] == -1).sum())
    return df

# --- Apply All Indicators ---
df = add_bollinger_bands(df)
df = add_macd(df)
df = add_rsi(df)
df = add_stochastic(df)
df = add_atr(df)
df = add_vwap(df)

# --- Apply All Signal Generators ---
df = signal_bollinger(df)
df = signal_macd(df)
df = signal_rsi(df)
df = signal_stochastic(df)
df = signal_atr(df)
df = signal_vwap(df)

# --- Display Required Tables ---

# Ensure index is datetime
df.index = pd.to_datetime(df.index)

# 📆 Last 5 Trading Days
print("📆 Last 5 Trading Days:")
last_5 = df.tail(5)[['Open', 'High', 'Low', 'Close',
                     'BB_SIGNAL', 'MACD_SIGNAL_FLAG', 'RSI_SIGNAL',
                     'STOCH_SIGNAL', 'ATR_SIGNAL', 'VWAP_SIGNAL']]
display(last_5)

# 📆 First 5 Trading Days After 3 Months
start_date = df.index.min()
cutoff = start_date + pd.DateOffset(months=3)
df_after_3_months = df[df.index > cutoff]

print("\n📆 First 5 Trading Days After 3 Months from Start:")
next_5 = df_after_3_months.head(5)[['Open', 'High', 'Low', 'Close',
                                    'BB_SIGNAL', 'MACD_SIGNAL_FLAG', 'RSI_SIGNAL',
                                    'STOCH_SIGNAL', 'ATR_SIGNAL', 'VWAP_SIGNAL']]
display(next_5)





Bollinger Bands buy signals: 38
Bollinger Bands sell signals: 50
MACD buy signals: 433
MACD sell signals: 318
RSI buy signals: 101
RSI sell signals: 95
Stochastic buy signals: 55
Stochastic sell signals: 64
ATR high volatility signals: 0
VWAP buy signals: 238
VWAP sell signals: 514
📆 Last 5 Trading Days:


,Open,High,Low,Close,BB_SIGNAL,MACD_SIGNAL_FLAG,RSI_SIGNAL,STOCH_SIGNAL,ATR_SIGNAL,VWAP_SIGNAL
Date,,,,,,,,,,
2024-12-23 00:00:00-05:00,22.730000,22.840000,22.510000,22.780001,0,-1,0,0,0,1
2024-12-24 00:00:00-05:00,22.580000,22.820000,22.549999,22.770000,0,-1,0,0,0,1
2024-12-26 00:00:00-05:00,22.639999,22.760000,22.600000,22.639999,0,-1,0,0,0,1
2024-12-27 00:00:00-05:00,22.590000,22.700001,22.299999,22.480000,0,-1,0,0,0,1
2024-12-30 00:00:00-05:00,22.219999,22.219999,21.889999,22.100000,0,-1,0,0,0,1



📆 First 5 Trading Days After 3 Months from Start:


,Open,High,Low,Close,BB_SIGNAL,MACD_SIGNAL_FLAG,RSI_SIGNAL,STOCH_SIGNAL,ATR_SIGNAL,VWAP_SIGNAL
Date,,,,,,,,,,
2022-04-04 00:00:00-04:00,22.676857,22.843056,22.658392,22.843056,0,-1,0,0,0,1
2022-04-05 00:00:00-04:00,22.621458,22.658392,22.409094,22.510658,0,-1,0,0,0,1
2022-04-06 00:00:00-04:00,22.076694,22.270594,21.919729,22.132093,0,-1,0,0,0,1
2022-04-07 00:00:00-04:00,21.928965,22.159796,21.855099,21.993597,0,-1,0,0,0,1
2022-04-08 00:00:00-04:00,21.956665,22.067463,21.827398,21.882799,1,-1,0,0,0,1
